In [1]:
import sys
import os
import datetime
import re
from pathlib import Path
from argparse import ArgumentParser
# modin does not work here due to certain df functions
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tinyhtml import html, h, frag, raw

In [2]:
def memstr_to_mebibyte(memstr: str) -> float:
    GIG = 1024.
    retval = 0.
    if memstr[-1] == 'm' or memstr[-1] == 'M':
        retval = float(memstr[:-1])
    elif memstr[-1] == 'g' or memstr[-1] == 'G':
        retval = float(memstr[:-1]) * GIG

    return retval

In [3]:
def parse_categories(cat):
    debug_p = False

    # takes the categories string and returns a dict in order to create new columns in the dataframe
    # we are interested only in the "-l bla bla" part

    # convert memory values to MiB
    # e.g. "-U non-deadlineusers -u zhanq -l A40=TRUE,gpu=1,gpu_A40=TRUE,h_stack=256m,h_vmem=4g,tmpfree=4g -pe threaded 32-48"
    #      -> {'gpu': 1, 'gpu_type': 'a40', 'h_stack': 256, 'h_vmem': 4*1024, 'tmpfree': 4*1024}

    if debug_p:
        print(f'DEBUG: parse_categories: type(cat) = {type(cat)}')
        print(f'DEBUG: parse_categories: cat = {cat}')

    resource_pat = re.compile(r'.*-l\ (\S*).*')
    if resource_pat.match(cat):
        resources = resource_pat.match(cat).groups(1)[0]
        if debug_p:
            print(f'DEBUG: parse_categories: resources = {resources}')
    else:
        #print(f'WARN: parse_categories: no resource request: cat = {cat}')
        #print()
        return None

    # dict to be returned
    retdict = {}

    # GPU types (complexes): A40, A100, P100, V100
    gpu_types = set(['a40', 'a100', 'p100', 'v100'])

    # we ignore the soft limits s_vmem, s_rt
    mem_types = set(['h_stack', 'h_vmem', 'tmpfree'])
    for tok in resources.split(','):
        key, val = tok.split('=')
        if key.lower() == 'gpu':
            # at some point, this became a boolean, then switched back to int
            retkey = key.lower()
            if val == 'TRUE':
                retval = 1
            else:
                retval = int(val)
        elif key.lower() in gpu_types:
            retkey = 'gpu_type'
            retval = key.lower()
        elif key.lower() in mem_types:
            retkey = key.lower()
            retval = memstr_to_mebibyte(val)
        elif key.lower() == 'hostname':
            # the accounting data already has a column "hostname" which is
            # the host allocated to the job
            retkey = 'requested_hostnames'
            retval = val
        elif key.lower() == 'h_rt':
            retkey = key.lower()
            # we have no time limit, so h_rt can be the string "INFINITY"
            # use 64-bit unsigned int
            if val.lower() == 'infinity':
                retval = np.iinfo(np.int64).max
            else:
                retval = np.int64(val)
        elif key.lower() == 'sgx':
            retkey = key.lower()
            retval = bool(val.lower() == 'true')
        else:
            retkey = None
            retval = None

        if retkey and retval:
            retdict[retkey] = retval
        else:
            continue

    # deal with missing values
    if 'h_rt' not in retdict:
        retdict['h_rt'] = np.iinfo(np.int64).max

    if 'gpu' not in retdict:
        retdict['gpu'] = 0
        retdict['gpu_type'] = None
    else:
        # gpu in retdict but gpu_type not specified
        if 'gpu_type' not in retdict:
            retdict['gpu_type'] = None

    if 'h_stack' not in retdict:
        retdict['h_stack'] = None

    if 'h_vmem' not in retdict:
        retdict['h_vmem'] = None

    if 'tmpfree' not in retdict:
        retdict['tmpfree'] = None

    if 'requested_hostnames' not in retdict:
        retdict['requested_hostnames'] = None

    if 'sgx' not in retdict:
        retdict['sgx'] = False

    if debug_p:
        print(f'DEBUG: parse_categories: retdict = {retdict}')

    return retdict

In [4]:
def prep_accounting(sgeacct_df, debug_p: bool):
    debug_p = False
    info_p = False
    # expand the df with stuff from the "-l resources_list"

    resources = sgeacct_df['category'].apply(parse_categories)

    if info_p:
        print(f'INFO: prep_accounting: type(resources) = {type(resources)}')

    #resources_df = pd.DataFrame(resources.tolist(), index=resources.index)
    resources_df = pd.DataFrame.from_records(resources.values, index=resources.index)

    if info_p:
        print(f'INFO: prep_accounting: sgeacct_df.describe() = \n{sgeacct_df.describe()}')
        print(f'INFO: prep_accounting: sgeacct_df.columns = \n{sgeacct_df.columns}')
        print(f'INFO: prep_accounting: sgeacct_df.head() = \n{sgeacct_df.head()}')

        print(f'INFO: prep_accounting: resources_df.describe() = \n{resources_df.describe()}')
        print(f'INFO: prep_accounting: resources_df.columns = \n{resources_df.columns}')
        print(f'INFO: prep_accounting: resources_df.head() = \n{resources_df.head()}')

    ret_df = pd.concat([sgeacct_df, resources_df], axis=1)

    if info_p:
        print( 'INFO: prep_accounting: after pd.concat()')
        print(f'INFO: prep_accounting: ret_df.columns = \n{ret_df.columns}')
        print(f'INFO: prep_accounting: ret_df.head() = \n{ret_df.head()}')

    return ret_df

In [42]:
acctpostproc = Path('accounting_postprocessed.feather')
sgeacct_df = pd.read_feather(acctpostproc)

In [43]:
# don't need the 'category' column - only useful for checking the accounting_postprocessed file
sgeacct_df.drop('category', axis=1, inplace=True)

In [44]:
sgeacct_df.describe()

,job_number,submission_time,start_time,end_time,failed,exit_status,ru_wallclock,ru_utime,slots,cpu,mem,io,iow,maxvmem,wait_time,h_stack,h_vmem,tmpfree,h_rt,gpu
count,1.470866e+07,14708657,14708657,14708657,1.470866e+07,1.470866e+07,1.470866e+07,1.470866e+07,1.470866e+07,1.470866e+07,1.470866e+07,1.470866e+07,14708657.0,1.470866e+07,14708657,1.470866e+07,1.470866e+07,1.463550e+07,1.470866e+07,1.470866e+07
mean,5.072637e+06,2023-10-02 11:24:32.969117184,2023-10-02 13:55:10.390292480,2023-10-02 14:32:58.086723584,1.143062e+00,4.623122e+00,2.250353e+03,2.143146e+03,1.177828e+00,2.795095e+03,3.855893e+04,3.758340e+01,0.0,3.409469e+09,0 days 02:30:37.421171627,2.565470e+02,2.179949e+04,7.833603e+03,9.014113e+18,4.254093e-03
min,1.000000e+00,2023-01-01 06:34:50,2023-01-01 06:45:40,2023-01-01 06:54:11,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0 days 00:00:01,8.000000e+00,8.000000e+02,1.000000e+01,1.000000e+00,0.000000e+00
25%,3.423273e+06,2023-07-17 06:10:40,2023-07-17 07:09:46,2023-07-17 07:15:15,0.000000e+00,0.000000e+00,2.600000e+01,1.588462e+01,1.000000e+00,2.109288e+01,1.075428e+00,1.438570e-01,0.0,2.787574e+08,0 days 00:00:37,2.560000e+02,8.192000e+03,4.096000e+03,9.223372e+18,0.000000e+00
50%,5.060861e+06,2023-11-23 08:11:31,2023-11-23 08:11:48,2023-11-23 08:15:45,0.000000e+00,0.000000e+00,1.820000e+02,1.496613e+02,1.000000e+00,1.710985e+02,7.640747e+01,2.396400e+00,0.0,2.408378e+09,0 days 00:10:44,2.560000e+02,1.331200e+04,4.096000e+03,9.223372e+18,0.000000e+00
75%,6.892601e+06,2023-12-22 09:06:02,2023-12-22 12:37:25,2023-12-22 12:48:01,0.000000e+00,0.000000e+00,4.860000e+02,3.305236e+02,1.000000e+00,3.661962e+02,2.816458e+02,1.051077e+01,0.0,3.221565e+09,0 days 01:20:30,2.560000e+02,2.867200e+04,4.096000e+03,9.223372e+18,0.000000e+00
max,9.999999e+06,2024-03-13 16:15:44,2024-03-13 16:16:00,2024-03-13 16:28:06,1.000000e+02,2.550000e+02,3.309973e+06,1.663623e+07,7.200000e+01,1.669182e+07,1.718480e+10,9.597258e+05,0.0,2.525975e+12,43 days 18:55:38,1.228800e+04,3.072000e+06,3.072000e+05,9.223372e+18,2.000000e+00
std,2.481260e+06,NaN,NaN,NaN,9.509080e+00,2.671070e+01,2.346231e+04,2.918836e+04,1.337560e+00,3.650663e+04,1.852516e+07,1.043641e+03,0.0,1.135526e+10,0 days 12:24:50.209143438,1.067048e+02,2.525167e+04,2.291834e+04,1.373419e+18,6.745340e-02


In [45]:
# keep successful jobs
sgeacct_df = sgeacct_df[sgeacct_df['failed'] == 0]

In [46]:
sgeacct_df.describe()

,job_number,submission_time,start_time,end_time,failed,exit_status,ru_wallclock,ru_utime,slots,cpu,mem,io,iow,maxvmem,wait_time,h_stack,h_vmem,tmpfree,h_rt,gpu
count,1.444591e+07,14445913,14445913,14445913,14445913.0,1.444591e+07,1.444591e+07,1.444591e+07,1.444591e+07,1.444591e+07,1.444591e+07,1.444591e+07,14445913.0,1.444591e+07,14445913,1.444591e+07,1.444591e+07,1.437299e+07,1.444591e+07,1.444591e+07
mean,5.050893e+06,2023-10-03 02:17:26.059517696,2023-10-03 04:47:17.462299392,2023-10-03 05:20:55.099415808,0.0,2.280444e+00,1.999979e+03,2.119222e+03,1.173605e+00,2.521920e+03,3.630417e+04,3.275761e+01,0.0,3.347206e+09,0 days 02:29:51.402784095,2.565746e+02,2.171641e+04,7.788233e+03,9.021101e+18,3.708800e-03
min,1.000000e+00,2023-01-01 06:34:50,2023-01-01 06:45:40,2023-01-01 06:54:11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0 days 00:00:01,8.000000e+00,8.000000e+02,1.000000e+01,6.000000e+01,0.000000e+00
25%,3.408201e+06,2023-07-17 07:50:18,2023-07-17 10:01:47,2023-07-17 10:07:03,0.0,0.000000e+00,2.500000e+01,1.643051e+01,1.000000e+00,2.069060e+01,1.029109e+00,1.436290e-01,0.0,2.703483e+08,0 days 00:00:36,2.560000e+02,8.192000e+03,4.096000e+03,9.223372e+18,0.000000e+00
50%,5.052308e+06,2023-11-23 13:20:26,2023-11-23 13:20:48,2023-11-23 13:24:43,0.0,0.000000e+00,1.820000e+02,1.496135e+02,1.000000e+00,1.702664e+02,8.230061e+01,2.303815e+00,0.0,2.408186e+09,0 days 00:10:48,2.560000e+02,1.228800e+04,4.096000e+03,9.223372e+18,0.000000e+00
75%,6.863502e+06,2023-12-22 12:15:31,2023-12-22 16:13:06,2023-12-22 16:21:22,0.0,0.000000e+00,4.550000e+02,3.251468e+02,1.000000e+00,3.524831e+02,2.792719e+02,9.871756e+00,0.0,3.218256e+09,0 days 01:22:39,2.560000e+02,2.867200e+04,4.096000e+03,9.223372e+18,0.000000e+00
max,9.999999e+06,2024-03-13 16:15:44,2024-03-13 16:16:00,2024-03-13 16:28:06,0.0,2.550000e+02,2.615081e+06,1.663623e+07,7.200000e+01,1.669182e+07,1.718480e+10,9.597258e+05,0.0,2.525975e+12,42 days 05:58:33,1.228800e+04,3.072000e+06,3.072000e+05,9.223372e+18,2.000000e+00
std,2.475713e+06,NaN,NaN,NaN,0.0,2.036583e+01,2.162091e+04,2.917978e+04,1.308033e+00,3.299564e+04,1.867454e+07,9.574569e+02,0.0,9.819477e+09,0 days 12:20:37.136821304,1.076503e+02,2.485988e+04,2.268400e+04,1.350816e+18,6.296619e-02


In [47]:
sgeacct_df.head(5)

,qname,hostname,owner,job_number,submission_time,start_time,end_time,failed,exit_status,ru_wallclock,...,maxvmem,wait_time,h_stack,h_vmem,tmpfree,h_rt,gpu,gpu_type,requested_hostnames,sgx
0,all.q,2118ffn004.bicic.local,buckovab,3084282,2023-01-01 06:43:03,2023-01-01 06:53:50,2023-01-01 06:54:11,0,0,21,...,639201280.0,0 days 00:10:47,256.0,204800.0,4096.0,9223372036854775807,0,None,None,False
1,all.q,2119fmn015.bicic.local,hwangg,3084293,2023-01-01 07:23:09,2023-01-01 07:33:31,2023-01-01 07:33:49,0,0,18,...,343056384.0,0 days 00:10:22,256.0,8192.0,10.0,172800,0,None,None,False
2,all.q,2119fmn021.bicic.local,hwangg,3084290,2023-01-01 07:23:04,2023-01-01 07:33:31,2023-01-01 07:33:50,0,0,19,...,342974464.0,0 days 00:10:27,256.0,8192.0,10.0,172800,0,None,None,False
3,all.q,2118ffn010.bicic.local,hwangg,3084292,2023-01-01 07:23:07,2023-01-01 07:33:31,2023-01-01 07:33:50,0,0,19,...,343171072.0,0 days 00:10:24,256.0,8192.0,10.0,172800,0,None,None,False
4,all.q,2118ffn009.bicic.local,hwangg,3084294,2023-01-01 07:23:10,2023-01-01 07:33:31,2023-01-01 07:33:50,0,0,19,...,343048192.0,0 days 00:10:21,256.0,8192.0,10.0,172800,0,None,None,False


In [48]:
sgeacct_df['ru_wallclock'] = pd.to_timedelta(sgeacct_df['ru_wallclock'], unit='s')
sgeacct_df['ru_utime'] = pd.to_timedelta(sgeacct_df['ru_utime'], unit='s')

In [49]:
gpu_a100_jobs_df = sgeacct_df[sgeacct_df['gpu_type'] == 'a100']

In [50]:
gpu_a100_jobs_df.describe()

,job_number,submission_time,start_time,end_time,failed,exit_status,ru_wallclock,ru_utime,slots,cpu,mem,io,iow,maxvmem,wait_time,h_stack,h_vmem,tmpfree,h_rt,gpu
count,1.688000e+03,1688,1688,1688,1688.0,1688.000000,1688,1688,1688.000000,1.688000e+03,1.688000e+03,1688.000000,1688.0,1.688000e+03,1688,1688.0,1688.000000,1688.0,1.688000e+03,1688.000000
mean,7.590149e+06,2023-10-22 04:51:52.483412480,2023-10-22 21:56:17.998222848,2023-10-23 02:10:25.626184960,0.0,3.332938,0 days 04:13:59.781990521,0 days 04:48:39.385040943,3.352488,2.130251e+04,1.271606e+05,1612.710100,0.0,1.121388e+10,0 days 17:04:25.514810426,256.0,68071.127962,4096.0,4.835713e+18,1.280213
min,2.037080e+05,2023-04-10 18:39:29,2023-04-10 18:43:35,2023-04-10 18:45:06,0.0,0.000000,0 days 00:00:00,0 days 00:00:00,1.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000e+00,0 days 00:00:02,256.0,1024.000000,4096.0,1.800000e+02,1.000000
25%,5.871737e+06,2023-09-22 19:50:55,2023-09-25 05:24:12,2023-09-25 06:04:32.750000128,0.0,0.000000,0 days 00:01:27.750000,0 days 00:00:15.396341,1.000000,2.431618e+01,2.293798e+01,0.044772,0.0,3.967293e+09,0 days 00:00:23,256.0,21504.000000,4096.0,6.000000e+02,1.000000
50%,8.817318e+06,2023-10-23 15:13:05,2023-10-24 02:39:47,2023-10-24 02:45:23,0.0,0.000000,0 days 00:05:35,0 days 00:00:16.169686,1.000000,2.665602e+01,3.744443e+01,0.178834,0.0,4.995402e+09,0 days 00:04:40.500000,256.0,98304.000000,4096.0,9.223372e+18,1.000000
75%,9.316132e+06,2023-11-21 16:13:06,2023-11-21 16:13:27.500000,2023-11-21 16:14:21.750000128,0.0,1.000000,0 days 00:06:13.250000,0 days 00:02:38.048570750,8.000000,2.062471e+02,1.034759e+03,7.471066,0.0,1.144481e+10,0 days 09:17:15,256.0,98304.000000,4096.0,9.223372e+18,2.000000
max,9.914222e+06,2024-03-12 18:35:40,2024-03-12 18:39:00,2024-03-12 20:15:09,0.0,255.000000,19 days 09:36:20,136 days 00:03:58.397622,64.000000,1.214300e+07,1.444153e+07,959725.799626,0.0,3.123186e+11,36 days 06:01:35,256.0,425984.000000,4096.0,9.223372e+18,2.000000
std,2.543896e+06,NaN,NaN,NaN,0.0,20.633863,0 days 23:05:22.441957029,3 days 09:23:09.393485792,3.570218,3.094263e+05,8.354732e+05,32045.479934,0.0,1.971972e+10,2 days 04:56:01.967428716,0.0,57029.543795,0.0,4.607606e+18,0.449236


In [51]:
gpu_a100_jobs_df.head(5)

,qname,hostname,owner,job_number,submission_time,start_time,end_time,failed,exit_status,ru_wallclock,...,maxvmem,wait_time,h_stack,h_vmem,tmpfree,h_rt,gpu,gpu_type,requested_hostnames,sgx
1637971,all.q,2119ga002.bicic.local,bergman,4401088,2023-04-10 18:39:29,2023-04-10 18:43:35,2023-04-10 18:45:06,0,0,0 days 00:01:31,...,47964160.0,0 days 00:04:06,256.0,4096.0,4096.0,420,1,a100,None,False
1637983,all.q,2119ga002.bicic.local,bergman,4401089,2023-04-10 18:39:30,2023-04-10 18:45:35,2023-04-10 18:47:25,0,0,0 days 00:01:50,...,63643648.0,0 days 00:06:05,256.0,4096.0,4096.0,420,1,a100,None,False
1637990,all.q,2119ga002.bicic.local,bergman,4401090,2023-04-10 18:39:30,2023-04-10 18:47:35,2023-04-10 18:48:25,0,0,0 days 00:00:50,...,53370880.0,0 days 00:08:05,256.0,4096.0,4096.0,420,1,a100,None,False
1638001,all.q,2119ga002.bicic.local,bergman,4401091,2023-04-10 18:39:30,2023-04-10 18:48:34,2023-04-10 18:50:05,0,0,0 days 00:01:31,...,63639552.0,0 days 00:09:04,256.0,4096.0,4096.0,420,1,a100,None,False
1638008,all.q,2119ga002.bicic.local,bergman,4401092,2023-04-10 18:39:30,2023-04-10 18:50:34,2023-04-10 18:51:45,0,0,0 days 00:01:11,...,62283776.0,0 days 00:11:04,256.0,4096.0,4096.0,420,1,a100,None,False


In [52]:
gpu_a100_jobs_df['ru_wallclock'].describe()

count                         1688
mean     0 days 04:13:59.781990521
std      0 days 23:05:22.441957029
min                0 days 00:00:00
25%         0 days 00:01:27.750000
50%                0 days 00:05:35
75%         0 days 00:06:13.250000
max               19 days 09:36:20
Name: ru_wallclock, dtype: object

In [53]:
gpu_a40_jobs_df = sgeacct_df[sgeacct_df['gpu_type'] == 'a40']

In [54]:
gpu_a40_jobs_df['ru_wallclock'].describe()

count                         3474
mean     0 days 09:24:35.046344271
std      1 days 06:23:12.234830303
min                0 days 00:00:00
25%         0 days 00:01:15.250000
50%                0 days 00:08:19
75%         0 days 13:36:00.750000
max               25 days 01:15:02
Name: ru_wallclock, dtype: object